In [167]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
from scipy import constants as const
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# File management #

In [173]:
data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")

In [174]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]
path=data.iloc[:,10]
Zapproach_fil=data.iloc[:,1]

In [177]:
#dat=[ 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]
dat=[ 0,1,2,3,4,5,6,7, 8, 11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 24,32]
T=[]
Tfiles=[]
Bfiles=[]
Jfiles=[]
size=[]
height=[]
Bfiles=[]
B=[]
for j in dat:
    #T.append([float(i) for i in temp[j].split(',')])
    #Tfiles.append([path[j]+i for i in temp_files[j].split(',')])
    size.append(size_dat[j])
    height.append(height_dat[j])
    B.append([int(i) for i in Bfields[j].split(',')])
    Bfiles.append([path[j]+i for i in Bfield_files[j].split(',')])
    Jfiles.append([path[j]+i for i in josephson[j].split(',')])


for j in range(len(B)):
    list1, list2 = zip(*sorted(zip(B[j], Bfiles[j])))
    B[j]=np.array(list1)
    Bfiles[j]=np.array(list2)
size=np.array(size)
height=np.array(height)

Zapproach=[]
Zapproach.append([path[32]+i for i in Zapproach_fil[32].split(',')])

# Functions

In [10]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    return I/np.abs(I[0])

def conductance(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    I=-np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    I=-np.gradient(I)
    return I/np.abs(I[0])

def gap_size(x,y,epsilon):
    xx=[]
    for i in range(len(y)):
        if np.abs(y[i])<epsilon:
            xx.append(x[i])
    if len(xx)<2:
        return 0,0
    else:
        return np.abs(xx[0]-xx[-1]), np.abs(xx[0]+xx[-1])/2

def gap_size2(x,y,epsilon):
    xx=[]
    half=int(len(y)/2)
    for i in range(half):
        if np.abs(y[half+i])<epsilon:
            xx.append(x[half+i])
        if np.abs(y[half-i])<epsilon:
            xx.append(x[half-i])
        else:
            break
    if len(xx)<2:
        return 0,0
    #elif np.abs(1-epsilon/min(y))<0.1:
    #    return 0,0
    else:
        return np.abs(max(xx)-min(xx)), np.abs(max(xx)+min(xx))/2


def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

# Analysis for the B field data

## 1st method: fitting the current

In [ ]:
for i in range(0,13):
    print(i)
    h=0
    cutt=np.arange(10,200,20)
    B800=B[i].index(800)
    R=[]
    R2=[]
    for cut in cutt:
        C1=[]
        Q0=[]
        erC=[]
        erQ0=[]
        for j in Bfiles[i][B800:-1]:
            spectra.load(j)
            x=np.array(spectra.bias)[200-cut:200+cut]
            y=np.array(spectra.current)[200-cut:200+cut]
            y=y/np.abs(y[0])
            #plt.plot(1000*x,y+h,linewidth=2)
            par,error=sc.optimize.curve_fit(lambda xx, C1,Q0: current(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[15.0,0.5]))
            #a=current2(1000*x,1,10000,par[0],0.0001,par[1],0.1)
            #plt.plot(1000*x,a+h)
            #h+=1
            #C1.append(par[0])
            #Q0.append(par[1])
            erC.append(np.sqrt(np.diag(error))[0])
            erQ0.append(np.sqrt(np.diag(error))[1])
        #plt.plot(1/np.array(C1))
        R.append(np.mean(erC))
        R2.append(np.mean(erQ0))
        #plt.plot(erQ0)
    plt.plot(cutt,R)


cut>100 doe snot change much the error

## 2nd method: fitting the conductance

In [ ]:
for i in range(0,13):
    print(i)
    h=0
    cutt=np.arange(50,100,10)
    B800=B[i].index(800)
    R=[]
    R2=[]
    for cut in cutt:
        C1=[]
        Q0=[]
        erC=[]
        erQ0=[]
        for j in Bfiles[i][B800:-1]:
            spectra.load(j)
            x=np.array(spectra.bias)[200-cut:200+cut]
            y=np.array(spectra.conductance)[200-cut:200+cut]
            y=y/np.abs(y[0])
            plt.plot(1000*x,y+1*h,linewidth=2)
            par,error=sc.optimize.curve_fit(lambda xx, C1,Q0: conductance(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[2.0,0.5]))
            a=current2(1000*x,1,10000,par[0],0.0001,par[1],0.1)
            plt.plot(1000*x,a+1*h)
            h+=1
            C1.append(par[0])
            Q0.append(par[1])
            erC.append(np.sqrt(np.diag(error))[0])
            erQ0.append(np.sqrt(np.diag(error))[1])
        R.append(np.mean(erC))
        R2.append(np.mean(erQ0))
    plt.plot(cutt,R)


The fit does not converge for all values of cut 

## 3rd method: calculate the zero current

In [ ]:
i=3
B800=B[i].index(800)
cut=np.arange(10,500,50)
for epsilon in cut:
    Eg=[]
    for j in Bfiles[i][B800:-1]:
        spectra.load(j)
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        y=y/np.abs(y[0])
        Eg.append(gap_size(x,y,max(y)/epsilon))
    plt.plot(Eg)


For values of epsilon bigger than 100 Eg is always linerarly decaying

In [ ]:
i=3
B800=B[i].index(800)
cut=np.arange(2,20,1)
for epsilon in cut:
    Eg=[]
    for j in Bfiles[i][B800:-1]:
        spectra.load(j)
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=y/np.abs(y[0])
        Eg.append(gap_size(x,y,max(y)/epsilon))
    plt.plot(Eg)

## Comparison between methods

In [ ]:
i=0
h=0

B800=B[i].index(800)

C1=[]
C2=[]
Eg1=[]
Eg2=[]
cut=200
epsilon1=50
epsilon2=2
for j in Bfiles[i][B800:-1]:
    spectra.load(j)
    x=np.array(spectra.bias)[200-cut:200+cut]
    y=np.array(spectra.current)[200-cut:200+cut]
    y=y/np.abs(y[0])
    par1,error1=sc.optimize.curve_fit(lambda xx, C1,Q0: current(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[15.0,0.5]))
    Eg1.append(gap_size(1000*x,y,max(y)/epsilon1))
    y=np.array(spectra.conductance)[200-cut:200+cut]
    y=y/np.abs(y[0])
    #plt.plot(1000*x,y)
    par2,error2=sc.optimize.curve_fit(lambda xx, C1,Q0: conductance(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[2.0,0.5]))
    Eg2.append(gap_size(1000*x,y,max(y)/epsilon2))
    C1.append(par1[0])
    C2.append(par2[0])
plt.plot(1/np.array(C1),label='current')
plt.plot(1/np.array(C2),label='conductance')
plt.plot(Eg1,label='gap1')
plt.plot(Eg2,label='gap2')
plt.legend()



All methods give a similar answer, excep the gap_size using the current, with depends twoo much on epsilon. It looks like themost relaiable one is the gap_size method using the conductance.

## New method: two coulomb gaps separated by E_zemman

In [150]:
i=4
h=0
B800=B[i].index(800)
cut=100
C1=[]
Q0=[]
R0=[]
Eg=[]
epsilon=2
for j in Bfiles[i][B800:-1]:
    spectra.load(j)
    x=np.array(spectra.bias)[200-cut:200+cut]
    y=np.array(spectra.conductance)[200-cut:200+cut]
    y=y/np.abs(y[0])
    #plt.plot(1000*x,y+1*h,linewidth=2)
    par,error=sc.optimize.curve_fit(lambda xx, C1,Q0: conductance(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[3.0,0.5]))
    a=current2(1000*x,1,10000,par[0],0.0001,par[1],0.1)
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.abs(y[0])
    Eg.append(gap_size(1000*x,y,max(y)/epsilon)[0])
    #plt.plot(1000*x,a+1*h)
    h+=1
    C1.append(par[0])
    R0.append(np.sqrt(np.diag(error))[0])


#plt.errorbar(B[i][B800:-1],C1,yerr=R0)
plt.plot(1/np.array(C1))
plt.plot(Eg)

## Analysis of the data using gap_size method with conductance

### Fisrt Method

In [879]:
EG=[]
V0=[]
for i in range(0,20):
    #B800=B[i].index(0)
    cut=100
    Eg=[]
    v00=[]
    #Epi=np.linspace(1.8,2.2,50)
    Epi=[2]
    h=0
    #for j in Bfiles[i][B800:-1]:
    for j in Bfiles[i]:
        Egt=[]
        v0t=[]
        for epsilon in Epi:
            spectra.load(j)
            x=np.array(spectra.bias)
            y=np.array(spectra.conductance)
            y=y/np.abs(y[0])
            #plt.plot(x,y+h)
            a,b=gap_size2(1000*x,y,np.max(y)/epsilon)
            Egt.append(a)
            v0t.append(b)
        h+=1
        Eg.append(np.mean(Egt))
        v00.append(np.mean(v0t))
    EG.append(Eg)
    V0.append(v00)

In [881]:
a=[]
b=[]
R1=[]
R2=[]
for i in range(0,20):
    B800=np.where(B[i]==800)[0][0]
    yt=EG[i][B800:-1]
    xt=B[i][B800:-1]
    y=[]
    x=[]
    for j in range(len(xt)):
        if np.abs(yt[j])>0.1:
            x.append(xt[j])
            y.append(yt[j])
        else:
            continue
    x=np.array(x)/1000
    y=np.array(y)
    par,err=sc.optimize.curve_fit(lambda xx, a,b: a*xx+b, x, y,p0=[-0.1,1],bounds=([-10,0.0],[0.0,10]))
    a.append(par[0])
    b.append(par[1])
    R1.append(np.sqrt(np.diag(err))[0])
    R2.append(np.sqrt(np.diag(err))[0])
    #plt.plot(x,y)
    #plt.plot(x,par[0]*x+par[1])

#plt.scatter(size,a/np.array(size))
#R=np.sqrt((R1/size)**2+(R1*10/size**2)**2)
#plt.errorbar(size,a/size,R,fmt="o")
#np.mean(a)/2
#np.mean(a/(size[0:20]*height[0:20]))
#t=a
#par,err=sc.optimize.curve_fit(lambda xx, a,b: a*xx+b, size[0:20],t,p0=[-1,0],bounds=([-10,0.0],[0.0,10]))
#plt.plot(size[0:20],par[0]*size[0:20]+par[1])
plt.scatter(size[0:20],a)
#par[0]


### Second Method

In [305]:
def gap_size3(x,y,epsilon):
    xx=[]
    half=int(len(y)/2)
    for i in range(half):
        if np.abs(y[half+i])<epsilon:
            xx.append(x[half+i])
        else:
            break
    for i in range(half):
        if np.abs(y[half-i])<epsilon:
            xx.append(x[half-i])
        else:
            break
    if len(xx)<2:
        return 0,0,x[half],x[half]
    #elif np.abs(1-epsilon/min(y))<0.1:
    #    return 0,0
    else:
        return np.abs(max(xx)-min(xx)), np.abs(max(xx)+min(xx))/2,max(xx),min(xx)



EG=[]
V0=[]
Epi=np.linspace(1.9,2.1,20)
for i in range(0,20):
    Eg=[]
    v00=[]
    xx=[]
    yy=[]
    h=0
    cut1=0
    spectra.load(Bfiles[i][0])
    cut2=len(spectra.bias)
    for j in Bfiles[i]:
        spectra.load(j)
        x=1000*np.array(spectra.bias)[cut1:cut2]
        y=np.array(spectra.conductance)[cut1:cut2]
        y=(y-min(y))/np.abs(y[0])
        #plt.plot(x,y+h)
        G=[]
        for ep in Epi:
            g,q0,a,b=gap_size3(x,y,np.max(y)/ep)    
            G.append(g)   
        Eg.append(np.mean(G))   
        x=1000*np.array(spectra.bias)
        y=np.array(spectra.conductance)
        #plt.scatter(x[np.where(x==a)[0][0]],y[np.where(x==a)[0][0]]+h)
        #plt.scatter(x[np.where(x==b)[0][0]],y[np.where(x==b)[0][0]]+h)
        cut1=min(np.where(x==a)[0][0],np.where(x==b)[0][0])-10
        cut2=max(np.where(x==a)[0][0],np.where(x==b)[0][0])+10
        h+=1
    EG.append(Eg)


In [271]:
for i in range(len(size)-1):
    plt.scatter(size[i]*height[i],EG[i][0],color='C0',label='B=0T')
    B800=np.where(B[i]==200)[0][0]
    plt.scatter(size[i]*height[i],EG[i][B800],color='C1',label='B=0.2T')
    B800=np.where(B[i]==400)[0][0]
    plt.scatter(size[i]*height[i],EG[i][B800],color='C2',label='B=0.4T')
    B800=np.where(B[i]==800)[0][0]
    plt.scatter(size[i]*height[i],EG[i][B800],color='C3',label='B=0.8T')
plt.legend()
plt.xlabel('Volume ($nm^3$)')
plt.ylabel('Gap ($meV$)')

Text(0, 0.5, 'Gap ($meV$)')

In [306]:

plt.scatter(B[8]/1000,EG[8],color='C0',label=str(int(size[8]*height[10]))+'$nm^3$') 
plt.scatter(B[10]/1000,EG[10],color='C1',label=str(int(size[10]*height[10]))+'$nm^3$') 
plt.scatter(B[12]/1000,EG[12],color='C2',label=str(int(size[12]*height[12]))+'$nm^3$') 
plt.scatter(B[14]/1000,EG[14],color='C3',label=str(int(size[14]*height[14]))+'$nm^3$') 
plt.legend()
plt.xlabel('B(T)')
plt.ylabel('Gap ($meV$)')

Text(0, 0.5, 'Gap ($meV$)')

In [293]:
ax1 = plt.subplot2grid((2,4), (0,0), colspan=2)
ax2 = plt.subplot2grid((2,4), (0,2), colspan=2)
ax3 = plt.subplot2grid((2,4), (1,0), colspan=2)
ax4 = plt.subplot2grid((2,4), (1,2), colspan=2)
ax=[ax1,ax2,ax3,ax4]
C=['C0','C1','C2','C3']
h=0
a=[8,10,12,14]
for i in range(len(a)):
    for j in Bfiles[a[i]]:
        spectra.load(j)
        x=1000*np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=(y-min(y))/np.abs(y[0])
        ax[i].plot(x,y+0.2*h,color=C[i],linewidth=2)
        h+=1

In [279]:
a=[]
b=[]
R1=[]
R2=[]
for i in range(0,20):
    B800=np.where(B[i]==800)[0][0]
    yt=EG[i][B800:-1]
    xt=B[i][B800:-1]
    y=[]
    x=[]
    for j in range(len(xt)):
        if np.abs(yt[j])>0.01:
            x.append(xt[j])
            y.append(yt[j])
        else:
            continue
    x=np.array(x)/1000
    y=np.array(y)
    par,err=sc.optimize.curve_fit(lambda xx, a,b: a*xx+b, x, y,p0=[-0.1,1],bounds=([-10,0.0],[0.0,10]))
    a.append(par[0])
    b.append(par[1])
    R1.append(np.sqrt(np.diag(err))[0])
    R2.append(np.sqrt(np.diag(err))[0])
    #plt.scatter(x,y)
    #plt.plot(x,par[0]*x+par[1])

#plt.scatter(size,a/np.array(size))
#R=np.sqrt((R1/size)**2+(R1*10/size**2)**2)
#plt.errorbar(size,a/size,R,fmt="o")
#np.mean(a)/2
plt.scatter(size[0:20],a)
#t=a
#par,err=sc.optimize.curve_fit(lambda xx, a,b: a*xx+b, size[0:20],t,p0=[-1,0],bounds=([-10,0.0],[0.0,10]))
#plt.plot(size[0:20],par[0]*size[0:20]+par[1])
#plt.scatter(size[0:20],a)
#par[0]

In [295]:
Eg=[]
v00=[]
i=8
for j in Bfiles[i]:
    Egt=[]
    v0t=[]
    Epi=[2]
    for epsilon in Epi:
        spectra.load(j)
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=y/np.abs(y[0])
        #plt.plot(1000*x,y)
        a,b=gap_size2(1000*x,y,np.max(y)/epsilon)
        Egt.append(a)
        v0t.append(b)
    Eg.append(np.mean(Egt))
    v00.append(np.mean(v0t))
plt.scatter(B[i],Eg,color='C0')        

### Only for the big Island

In [149]:
i=20
cut=100
Eg=[]
v00=[]
#Epi=np.linspace(1.8,2.2,50)
Epi=[2]
h=0
#for j in Bfiles[i][B800:-1]:
for j in Bfiles[i]:
    Egt=[]
    v0t=[]
    for epsilon in Epi:
        spectra.load(j)
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=y/np.abs(y[0])
        y=y
        y=sc.signal.savgol_filter(y,21,1)
        f=sc.interpolate.interp1d(x,y)
        xx=np.linspace(-0.005,0.005,10000)
        plt.plot(x,(y-1)/10+h)
        yy=np.abs(np.gradient(f(xx)))
        plt.plot(xx,yy+h)
        peak=sc.signal.find_peaks(yy/max(yy),width=20,prominence=0.14)[0]
        plt.scatter(xx[peak],yy[peak]+h)
        if len(peak)>=2:
            temp=np.sort(np.abs(xx[peak]))

            Eg.append(temp[0]+temp[1])
        else:
            Eg.append(0)
    h+=1

#print(Eg)
#plt.scatter(B[i],Eg)

In [242]:
d=2.1
par,err=sc.optimize.curve_fit(lambda xx, a,b,c: d*np.real(np.sqrt(1-((xx+0.0001*1j)/a)**2))+b*np.real(np.sqrt(1-((xx+0.0001*1j)/c)**2)), B[20]/1000,1000*np.array(Eg) ,p0=[0.7,2.6,0.2],bounds=([0.0,1.0,0.1],[0.8,5,0.8]))
plt.scatter(B[20]/1000,1000*np.array(Eg))
x=np.linspace(0,0.8,100)
plt.plot(x,  d*np.real(np.sqrt(1-((x+0.0001*1j)/par[0])**2))+par[1]*np.real(np.sqrt(1-((x+0.0001*1j)/par[2])**2))  )
par[0],par[1],par[2]

(0.5972608212199684, 2.5435559499443166, 0.276071223790091)

# Analysis for the T dependent data

We can not use the same method we used earlier. The broadening due to temperature is too big to calculate the gap. It is better to fit the T=8.0K graph with the model

In [265]:
EG8K=[]
V0=[]
h=0
for i in range(0,12):
    cut=50

    j=Tfiles[i][0]
    Egt=[]
    v0t=[]
    #for cut in cutt:
    spectra.load(j)
    x=np.array(spectra.bias)
    half=int(len(x)/2)
    x=np.array(spectra.bias)[half-cut:half+cut]
    y=np.array(spectra.current)[half-cut:half+cut]
    #x=np.array(spectra.bias)
    #y=np.array(spectra.current)
    y=y/np.abs(y[0])
    #plt.plot(1000*x,y+h)
    par,err=sc.optimize.curve_fit(lambda xx, C1,Q0: current(xx, 1,10000, C1,0.001,Q0,0.7), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.1],[5.0,0.1]))
    EG8K.append(1/par[0])
    V0.append(par[1])
    #plt.plot(1000*x,current(1000*x, 1,10000, par[0],0.001,par[1],0.7)+h)
    h+=1
    #EG.append(np.mean(Egt))
    #V0.append(np.mean(v0t))

In [264]:
plt.scatter(size,EG8K)

# Probak